In [1]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ultralytics

   ---------------------------------------- 0.0/872.8 kB ? eta -:--:--
   -- ------------------------------------- 61.4/872.8 kB 3.2 MB/s eta 0:00:01
   -------- ------------------------------- 194.6/872.8 kB 2.3 MB/s eta 0:00:01
   -------------- ------------------------- 307.2/872.8 kB 2.4 MB/s eta 0:00:01
   ----------------- ---------------------- 389.1/872.8 kB 2.2 MB/s eta 0:00:01
   -------------------------- ------------- 583.7/872.8 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 872.8/872.8 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB 21.8 MB/s eta 0:00:02
   ---------------------------------------- 0.3/38.8 MB 21.8 MB/s eta 0:00:02
   ---------------------------------------- 0.4/38.8 MB 3.2 MB/s eta 0:00:13
   ---------------------------------------- 0.4/38.8 MB 3.2 MB/s eta 0:00:13
   - -------------------------------------- 1.1/38.8 MB 5.5 MB/s eta 0

In [4]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install boto3

   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.2 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------ 71.7/139.2 kB 975.2 kB/s eta 0:00:01
   ---------------------------------------- 139.2/139.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.5 MB 6.8 MB/s eta 0:00:02
    --------------------------------------- 0.2/12.5 MB 3.6 MB/s eta 0:00:04
   - -------------------------------------- 0.5/12.5 MB 3.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/12.5 MB 4.0 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/12.5 MB 5.5 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/12.5 MB 6.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/12.5 MB 6.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/12.5 MB 4.7 MB/s eta 0:00:03
   -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.35.20 which is incompatible.


In [117]:
import boto3

def get_file_name(s):
    index = s.rfind('/')
    if index != -1:
        return s[index + 1:]
    return s
    
def get_image(url):
    session = boto3.session.Session()
    access_key = 'YCAJEyQdZPmhZ9VGNzXafx6RD'
    secret_key = 'YCPk0adBo8_d-mYM9pBArJ48_WJ5YG8CyoMeXR9k'
    bucket_name = 'audiobacket1234a'  # Название вашего бакета
    file_key = url    # Путь к файлу в бакете
    local_file_name =  'D:\\' + url
    
    s3 = session.client(
        service_name='s3',
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        endpoint_url='https://storage.yandexcloud.net'
    )

    # Получить объект
    s3.download_file(bucket_name, url, local_file_name)

    return local_file_name


In [119]:
from ultralytics import YOLO

names = ['Перекрестный', 'Глубокий', 'Дистальный', 'Мезиальный', 'Нормальный', 'Открытый']

def get_result(results):
    probs = []
    for result in results:
        for i in range(len(result.probs)):
            probs.append(float(f'{result.probs[i].data:.3f}'))
    indx = probs.index(max(probs))
    return names[indx] + " " + str(round(probs[indx]*100, 2)) + "%"

def load_image(file_path):      
    if file_path:
        results = model.predict(file_path)  # predict on an image
        return get_result(results)

In [ ]:
from fastapi import FastAPI, Request, File
from pydantic import BaseModel
import os
import nest_asyncio
import uvicorn
import json

nest_asyncio.apply()

app = FastAPI()

model = YOLO("last.pt")

class PredictionRequest(BaseModel):
   input: str

@app.post("/predict")
async def predict(request: Request):
    json_data = await request.json()  # Получаем JSON из тела запроса
    url = json_data.get("url")
    if url:
        image_path = get_image(url)  # Сохраняем файл и получаем путь к нему
        result = load_image(image_path)  # Обрабатываем изображение
        return {"result": result}
    return {"error": "URL not provided"}

if __name__ == "__main__":
       config = uvicorn.Config(app, host="0.0.0.0", port=8000)
       server = uvicorn.Server(config)
       server.run()

INFO:     Started server process [27216]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Task exception was never retrieved
future: <Task finished name='Task-91' coro=<Server.serve() done, defined at D:\anaconda\Lib\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nbuttr\AppData\Local\Temp\ipykernel_27216\1528514777.py", line 30, in <module>
    server.run()
  File "D:\anaconda\Lib\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^


image 1/1 D:\photo.jpg: 640x640 mesial 0.92, cross 0.07, open 0.00, normal 0.00, distal 0.00, 89.2ms
Speed: 5.1ms preprocess, 89.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
INFO:     172.27.0.1:61196 - "POST /predict HTTP/1.1" 200 OK
INFO:     172.27.0.1:61854 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "D:\anaconda\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "D:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "D:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope, recei

INFO:     172.27.0.1:62307 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "D:\anaconda\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "D:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "D:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope, recei


image 1/1 D:\photo-42134827.jpg: 640x640 deep 0.74, normal 0.13, cross 0.12, open 0.01, distal 0.00, 102.0ms
Speed: 15.0ms preprocess, 102.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
INFO:     172.27.0.1:63370 - "POST /predict HTTP/1.1" 200 OK

image 1/1 D:\photo-1350939396.jpg: 640x640 mesial 0.92, cross 0.07, open 0.00, normal 0.00, distal 0.00, 101.3ms
Speed: 9.0ms preprocess, 101.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
INFO:     172.27.0.1:63813 - "POST /predict HTTP/1.1" 200 OK

image 1/1 D:\photo2079746377.jpg: 640x640 deep 0.74, normal 0.13, cross 0.12, open 0.01, distal 0.00, 87.7ms
Speed: 13.0ms preprocess, 87.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
INFO:     172.27.0.1:63875 - "POST /predict HTTP/1.1" 200 OK

image 1/1 D:\photo-284791412.jpg: 640x640 open 0.93, mesial 0.05, cross 0.02, distal 0.00, deep 0.00, 101.5ms
Speed: 11.4ms preprocess, 101.5ms inference, 0.0ms postprocess per image at s